In [ ]:
import numpy as np
import quagmire
from quagmire import tools as meshtools

In [ ]:
x, y, height = np.loadtxt("../../data/ETOPO1-Tasmania-proj.dat", unpack=True)

# filter out points we do not want

point_mask = height > -50.0

xs = x[point_mask]
ys = y[point_mask]
heights = height[point_mask]

points = np.column_stack([xs, ys])

submarine = heights < 0.0
subaerial = heights >= 0.0

In [ ]:
import lavavu
import stripy

tri = stripy.Triangulation(xs, ys)

lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[1000,600], near=-10.0)

tris = lv.triangles("mesh",  wireframe=False, colour="#77ff88", opacity=1.0)
tris.vertices(tri.points)
tris.indices(tri.simplices)
tris.values(-heights, label="elevation")
tris.colourmap('dem3')
cb = tris.colourbar()

sa = lv.points("subaerial", colour="red", pointsize=3, opacity=0.75)
sa.vertices(points[subaerial])
sm = lv.points("submarine", colour="blue", pointsize=3, opacity=0.75)
sm.vertices(points[submarine])

lv.control.Panel()
lv.control.ObjectList()
lv.control.show()

lv.show()

In [ ]:
DM = meshtools.create_DMPlex_from_points(xs, ys, submarine)
mesh = quagmire.SurfaceProcessMesh(DM)

In [ ]:
mesh.update_height(heights)

gradient_max = mesh.slope.max()
gradient_mean = mesh.slope.mean()
flat_spots = mesh.identify_flat_spots()
low_points = mesh.identify_low_points()

print("mean gradient {}\nnumber of flat spots {}\nnumber of low points {}".format(gradient_mean,
                                                                                  flat_spots.size,
                                                                                  low_points.shape[0])) 

In [ ]:
flowpaths1 = mesh.cumulative_flow(mesh.area)
flowpaths1 = mesh.rbf_smoother(flowpaths1)

In [ ]:
new_heights = mesh.low_points_local_patch_fill(its=2)
mesh._update_height_partial(new_heights)
low_points2 = mesh.identify_low_points()

print("number of low points {}".format(low_points2.size))

In [ ]:
new_heights = mesh.low_points_swamp_fill(ref_height=-0.01)

print new_heights.min()

In [ ]:
for i in range(0,10):
    new_heights = mesh.low_points_swamp_fill(ref_height=-0.01)
    mesh._update_height_partial(new_heights)
    # mesh.update_height(new_heights)
    low_points2 = mesh.identify_low_points()
    print low_points2.shape

In [ ]:
flowpaths2 = mesh.cumulative_flow(mesh.area)
flowpaths2 = mesh.rbf_smoother(flowpaths2)

In [ ]:
lv = lavavu.Viewer(border=False, background="#FFFFFF", resolution=[1500,1000], near=-10.0)


topo= lv.triangles("topography",  wireframe=False)
topo.vertices(mesh.tri.points)
topo.indices(mesh.tri.simplices)
topo.values(heights, 'topography')
topo.values(flowpaths1, 'flow1')
topo.values(flowpaths2, 'flow2')
topo["zmin"] = -0.05

# topo.colourmap(["(-1.0)#FFFFFF:0.0 (-0.01)#0099FF:0.3 (0.01)#FFFF99:1.0 (0.5)#33AA11:1.0 (2.5)#886644:1.0"] , logscale=False, range=[0.0,1.0])   # Apply a built in colourmap
# topo.colourmap(["(-1.0)#FFFFFF:1.0 (-0.01)#0099FF:1.0 (0.01)#AAAAAA:1.0 (0.5)#777777:1.0 (2.5)#555555:1.0"] , logscale=False)   # Apply a built in colourmap
topo.colourmap("terrain")
topo.colourbar()

lv.control.Panel()
lv.control.ObjectList()
topo.control.List(["topography", "flow1", "flow2"], property="colourby", value="orginal", command="redraw")
# flowpath2.control.List(["flow1", "flow2"], property="colourby", value="flow2", command="reload")
lv.control.show()